In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import statsmodels.api as sm

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_csv('/Users/rodney/Documents/Jupyter/HW_IE_691/HW_1/PCD_ALL.csv')
# print(df)

In [3]:
df = df.drop(columns=['train'])
# print(df)

In [4]:
X = df[['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']]
y = df['lpsa']

print (X)
print (y)

      lcavol   lweight  age      lbph  svi       lcp  gleason  pgg45
0  -0.579818  2.769459   50 -1.386294    0 -1.386294        6      0
1  -0.994252  3.319626   58 -1.386294    0 -1.386294        6      0
2  -0.510826  2.691243   74 -1.386294    0 -1.386294        7     20
3  -1.203973  3.282789   58 -1.386294    0 -1.386294        6      0
4   0.751416  3.432373   62 -1.386294    0 -1.386294        6      0
5  -1.049822  3.228826   50 -1.386294    0 -1.386294        6      0
6   0.737164  3.473518   64  0.615186    0 -1.386294        6      0
7   0.693147  3.539509   58  1.536867    0 -1.386294        6      0
8  -0.776529  3.539509   47 -1.386294    0 -1.386294        6      0
9   0.223144  3.244544   63 -1.386294    0 -1.386294        6      0
10  0.254642  3.604138   65 -1.386294    0 -1.386294        6      0
11 -1.347074  3.598681   63  1.266948    0 -1.386294        6      0
12  1.613430  3.022861   63 -1.386294    0 -0.597837        7     30
13  1.477049  2.998229   67 -1.386

# NOTE:
## The following performs a forward-backward feature selection based on p-value of each predictor/feature. The alpha is set to 0.05 for both the forward and the backward piecewise selection methods.
## The code returns a list of selected features.

In [5]:
def stepwise_selection(X, y, 
                       initial_list=[],         # list of features to start with (column names of X)
                       threshold_in=0.05,       # include a feature if its p-value < threshold_in
                       threshold_out=0.05,      # exclude a feature if its p-value > threshold_out
                       verbose=True):           # whether to print the sequence of inclusions and exclusions
   
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(X, y)

print('resulting features:')
print(result)

Add  lcavol                         with p-value 1.11861e-17
Add  lweight                        with p-value 0.000493799
Add  svi                            with p-value 0.00179762
resulting features:
['lcavol', 'lweight', 'svi']


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.


# INTERPRETATION OF RESULTS:
### The results show that most significant predictors/features to the model are 'lcavol', 'lweight', and 'svi'. These are chosen based off their p-values. 
### For the forward stepwise, predictors/feautres whose p-values were LESS than the alpha of 0.05 were ADDED to the subset.
### For the backward stepwise, predictors/features wholse p-values were GREATER than the alpha of 0.05 were REMOVED.